In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/building-density2-taller-than-18m-500m/valid_df_with_building_density2 (3).csv
/kaggle/input/building-density2-taller-than-18m-500m/train_df_with_building_density2 (3).csv
/kaggle/input/data-with-density-200m/validation_data_with_density.csv
/kaggle/input/data-with-density-200m/uncorrupted_training_data_with_density.csv
/kaggle/input/green-space-area-550m/valid_df_with_green_space (7).csv
/kaggle/input/green-space-area-550m/train_df_with_green_space (7).csv
/kaggle/input/new-road-density-450m/train_df_with_road_density.csv
/kaggle/input/new-road-density-450m/valid_df_with_road_density.csv
/kaggle/input/satellite-50m-buffer/validation_data_with_satellite.csv
/kaggle/input/satellite-50m-buffer/training_data_with_satellite.csv
/kaggle/input/new-water-750m/valid_df_with_water (14).csv
/kaggle/input/new-water-750m/train_df_with_water (14).csv


In [2]:
# Feature Engineering
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, QuantileTransformer, PowerTransformer, Normalizer
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score

import optuna

# Combine Datasets

In [3]:
train_density = pd.read_csv("/kaggle/input/data-with-density-200m/uncorrupted_training_data_with_density.csv")
train_density.head()

,Longitude,Latitude,datetime,UHI Index,density
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8


In [4]:
train_satellite = pd.read_csv("/kaggle/input/satellite-50m-buffer/training_data_with_satellite.csv")
train_satellite.head()

,Longitude,Latitude,datetime,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,992.949495,969.676768,1130.010101,1223.454545,1451.595960,1813.282828,1941.626263,1890.161616,1995.797980,1890.030303,1630.909091,0.214126,-0.000035,-0.251691,38.009627
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,988.989796,949.540816,1101.928571,1191.387755,1428.622449,1823.163265,1950.183673,1918.306122,2012.367347,1864.530612,1588.693878,0.233759,-0.014216,-0.270303,38.009627
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,967.108911,913.376238,1068.574257,1140.584158,1387.514851,1855.405941,1994.138614,1992.079208,2074.861386,1832.712871,1514.772277,0.271812,-0.041667,-0.301735,37.939344
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,969.100000,920.130000,1076.030000,1139.090000,1374.240000,1862.310000,2000.800000,2018.840000,2088.380000,1818.730000,1477.340000,0.278584,-0.052145,-0.304636,37.692563
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,964.711340,919.000000,1076.845361,1127.010309,1361.938144,1873.649485,2015.896907,2044.402062,2108.865979,1816.154639,1451.185567,0.289269,-0.059123,-0.309990,37.712901


In [5]:
train_satellite = train_satellite.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat = pd.concat([train_density, train_satellite], axis = 1)
train_concat.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,992.949495,969.676768,1130.010101,1223.454545,1451.595960,1813.282828,1941.626263,1890.161616,1995.797980,1890.030303,1630.909091,0.214126,-0.000035,-0.251691,38.009627
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,988.989796,949.540816,1101.928571,1191.387755,1428.622449,1823.163265,1950.183673,1918.306122,2012.367347,1864.530612,1588.693878,0.233759,-0.014216,-0.270303,38.009627
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,967.108911,913.376238,1068.574257,1140.584158,1387.514851,1855.405941,1994.138614,1992.079208,2074.861386,1832.712871,1514.772277,0.271812,-0.041667,-0.301735,37.939344
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,969.100000,920.130000,1076.030000,1139.090000,1374.240000,1862.310000,2000.800000,2018.840000,2088.380000,1818.730000,1477.340000,0.278584,-0.052145,-0.304636,37.692563
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,964.711340,919.000000,1076.845361,1127.010309,1361.938144,1873.649485,2015.896907,2044.402062,2108.865979,1816.154639,1451.185567,0.289269,-0.059123,-0.309990,37.712901


In [6]:
validation_density = pd.read_csv("/kaggle/input/data-with-density-200m/validation_data_with_density.csv")
validation_density.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,density
0,-73.971665,40.788763,NaN,12
1,-73.971928,40.788875,NaN,12
2,-73.967080,40.789080,NaN,4
3,-73.972550,40.789082,NaN,12
4,-73.969697,40.787953,NaN,9


In [7]:
validation_satellite = pd.read_csv("/kaggle/input/satellite-50m-buffer/validation_data_with_satellite.csv")
validation_satellite.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.971665,40.788763,NaN,927.510204,760.561224,922.836735,872.755102,1302.051020,2312.530612,2593.826531,2631.510204,2726.244898,1989.224490,1455.785714,0.501890,-0.139001,-0.480728,36.240387
1,-73.971928,40.788875,NaN,1020.693878,917.367347,1091.816327,1059.428571,1475.663265,2415.785714,2705.061224,2716.183673,2801.489796,2091.010204,1566.336735,0.438804,-0.130050,-0.426567,36.220483
2,-73.967080,40.789080,NaN,970.545455,1154.525253,1277.585859,1368.838384,1500.555556,1720.090909,1848.050505,1792.646465,1900.414141,1907.929293,1672.373737,0.134053,0.031153,-0.167759,35.535937
3,-73.972550,40.789082,NaN,1204.210000,1551.800000,1713.960000,1859.680000,2032.640000,2282.490000,2406.940000,2352.380000,2430.190000,2508.810000,2228.220000,0.116974,0.032179,-0.157001,36.614672
4,-73.969697,40.787953,NaN,1178.479592,1299.132653,1430.418367,1444.836735,1668.214286,2285.122449,2559.214286,2517.551020,2620.928571,2385.000000,1968.979592,0.270724,-0.027037,-0.275365,34.665793


In [8]:
validation_satellite = validation_satellite.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat = pd.concat([validation_density, validation_satellite], axis=1)
validation_concat.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.971665,40.788763,NaN,12,927.510204,760.561224,922.836735,872.755102,1302.051020,2312.530612,2593.826531,2631.510204,2726.244898,1989.224490,1455.785714,0.501890,-0.139001,-0.480728,36.240387
1,-73.971928,40.788875,NaN,12,1020.693878,917.367347,1091.816327,1059.428571,1475.663265,2415.785714,2705.061224,2716.183673,2801.489796,2091.010204,1566.336735,0.438804,-0.130050,-0.426567,36.220483
2,-73.967080,40.789080,NaN,4,970.545455,1154.525253,1277.585859,1368.838384,1500.555556,1720.090909,1848.050505,1792.646465,1900.414141,1907.929293,1672.373737,0.134053,0.031153,-0.167759,35.535937
3,-73.972550,40.789082,NaN,12,1204.210000,1551.800000,1713.960000,1859.680000,2032.640000,2282.490000,2406.940000,2352.380000,2430.190000,2508.810000,2228.220000,0.116974,0.032179,-0.157001,36.614672
4,-73.969697,40.787953,NaN,9,1178.479592,1299.132653,1430.418367,1444.836735,1668.214286,2285.122449,2559.214286,2517.551020,2620.928571,2385.000000,1968.979592,0.270724,-0.027037,-0.275365,34.665793


# Add features

## 1. Enhanced Vegetation Index (EVI)
The EVI is used to measure vegetation greenness and is an improvement over NDVI, as it is less sensitive to atmospheric conditions and soil background.

$$
\text{EVI} = 2.5 \cdot \frac{(B8 - B4)}{(B8 + 6 \cdot B4 - 7.5 \cdot B2 + 1)}
$$

Where:
- \(B8\) = Near-Infrared (NIR) band (Band 8)
- \(B4\) = Red band (Band 4)
- \(B2\) = Blue band (Band 2)
- The constants are: \( G = 2.5, C_1 = 6, C_2 = 7.5, L = 1 \)

## 2. Soil Adjusted Vegetation Index (SAVI)
The SAVI is an index used to measure vegetation while adjusting for the influence of soil brightness in areas with sparse vegetation.

$$
\text{SAVI} = \frac{(B8 - B4)}{(B8 + B4 + 0.5)} \cdot (1 + 0.5)
$$

Where:
- \(B8\) = Near-Infrared (NIR) band (Band 8)
- \(B4\) = Red band (Band 4)
- The constant \(L = 0.5\) is used for intermediate vegetation cover.

## 3. Modified Normalized Difference Water Index (MNDWI)
The MNDWI is used for detecting water bodies, making it particularly useful for water resource monitoring and flood detection.

$$
\text{MNDWI} = \frac{(B3 - B11)}{(B3 + B11)}
$$

Where:
- \(B3\) = Green band (Band 3)
- \(B11\) = Shortwave Infrared (SWIR1) band (Band 11)

In [9]:
# Calculate EVI
train_concat['EVI'] = 2.5 * (train_concat['B08'] - train_concat['B04']) / (train_concat['B08'] + 6 * train_concat['B04'] - 7.5 * train_concat['B02'] + 1)
validation_concat['EVI'] = 2.5 * (validation_concat['B08'] - validation_concat['B04']) / (validation_concat['B08'] + 6 * validation_concat['B04'] - 7.5 * validation_concat['B02'] + 1)

# Calculate SAVI
train_concat['SAVI'] = (train_concat['B08'] - train_concat['B04']) / (train_concat['B08'] + train_concat['B04'] + 0.5) * (1 + 0.5)
validation_concat['SAVI'] = (validation_concat['B08'] - validation_concat['B04']) / (validation_concat['B08'] + validation_concat['B04'] + 0.5) * (1 + 0.5)

# Calculate MNDWI
train_concat['MNDWI'] = (train_concat['B03'] - train_concat['B11']) / (train_concat['B03'] + train_concat['B11'])
validation_concat['MNDWI'] = (validation_concat['B03'] - validation_concat['B11']) / (validation_concat['B03'] + validation_concat['B11'])


In [10]:
train_concat.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,B8A,B11,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,992.949495,969.676768,1130.010101,1223.454545,1451.595960,...,1995.797980,1890.030303,1630.909091,0.214126,-0.000035,-0.251691,38.009627,0.850690,0.321138,-0.251659
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,988.989796,949.540816,1101.928571,1191.387755,1428.622449,...,2012.367347,1864.530612,1588.693878,0.233759,-0.014216,-0.270303,38.009627,0.933826,0.350582,-0.257075
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,967.108911,913.376238,1068.574257,1140.584158,1387.514851,...,2074.861386,1832.712871,1514.772277,0.271812,-0.041667,-0.301735,37.939344,1.071730,0.407653,-0.263379
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,969.100000,920.130000,1076.030000,1139.090000,1374.240000,...,2088.380000,1818.730000,1477.340000,0.278584,-0.052145,-0.304636,37.692563,1.125919,0.417810,-0.256567
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,964.711340,919.000000,1076.845361,1127.010309,1361.938144,...,2108.865979,1816.154639,1451.185567,0.289269,-0.059123,-0.309990,37.712901,1.197662,0.433835,-0.255551


In [11]:
validation_concat.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,B8A,B11,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI
0,-73.971665,40.788763,NaN,12,927.510204,760.561224,922.836735,872.755102,1302.051020,2312.530612,...,2726.244898,1989.224490,1455.785714,0.501890,-0.139001,-0.480728,36.240387,2.031053,0.752727,-0.366197
1,-73.971928,40.788875,NaN,12,1020.693878,917.367347,1091.816327,1059.428571,1475.663265,2415.785714,...,2801.489796,2091.010204,1566.336735,0.438804,-0.130050,-0.426567,36.220483,1.888255,0.658119,-0.313933
2,-73.967080,40.789080,NaN,4,970.545455,1154.525253,1277.585859,1368.838384,1500.555556,1720.090909,...,1900.414141,1907.929293,1672.373737,0.134053,0.031153,-0.167759,35.535937,0.786147,0.201048,-0.197878
3,-73.972550,40.789082,NaN,12,1204.210000,1551.800000,1713.960000,1859.680000,2032.640000,2282.490000,...,2430.190000,2508.810000,2228.220000,0.116974,0.032179,-0.157001,36.614672,0.657649,0.175440,-0.188230
4,-73.969697,40.787953,NaN,9,1178.479592,1299.132653,1430.418367,1444.836735,1668.214286,2285.122449,...,2620.928571,2385.000000,1968.979592,0.270724,-0.027037,-0.275365,34.665793,1.857094,0.406035,-0.250191


## 4. Green Space with 550m buffer

In [12]:
train_green = pd.read_csv("/kaggle/input/green-space-area-550m/train_df_with_green_space (7).csv")
train_green.head()

,Longitude,Latitude,datetime,UHI Index,green_space
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,185571.898005
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,189946.352368
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,194989.481631
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,200157.261231
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,204423.999904


In [13]:
validation_green = pd.read_csv("/kaggle/input/green-space-area-550m/valid_df_with_green_space (7).csv")
validation_green.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,green_space
0,-73.971665,40.788763,NaN,24499.889319
1,-73.971928,40.788875,NaN,10574.872312
2,-73.967080,40.789080,NaN,403763.833896
3,-73.972550,40.789082,NaN,5144.297626
4,-73.969697,40.787953,NaN,222084.003584


In [14]:
train_green = train_green.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat1 = pd.concat([train_concat, train_green], axis = 1)
train_concat1.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,B11,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,992.949495,969.676768,1130.010101,1223.454545,1451.595960,...,1890.030303,1630.909091,0.214126,-0.000035,-0.251691,38.009627,0.850690,0.321138,-0.251659,185571.898005
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,988.989796,949.540816,1101.928571,1191.387755,1428.622449,...,1864.530612,1588.693878,0.233759,-0.014216,-0.270303,38.009627,0.933826,0.350582,-0.257075,189946.352368
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,967.108911,913.376238,1068.574257,1140.584158,1387.514851,...,1832.712871,1514.772277,0.271812,-0.041667,-0.301735,37.939344,1.071730,0.407653,-0.263379,194989.481631
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,969.100000,920.130000,1076.030000,1139.090000,1374.240000,...,1818.730000,1477.340000,0.278584,-0.052145,-0.304636,37.692563,1.125919,0.417810,-0.256567,200157.261231
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,964.711340,919.000000,1076.845361,1127.010309,1361.938144,...,1816.154639,1451.185567,0.289269,-0.059123,-0.309990,37.712901,1.197662,0.433835,-0.255551,204423.999904


In [15]:
train_concat1.shape

(11229, 24)

In [16]:
validation_green = validation_green.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat1 = pd.concat([validation_concat, validation_green], axis = 1)
validation_concat1.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,B11,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space
0,-73.971665,40.788763,NaN,12,927.510204,760.561224,922.836735,872.755102,1302.051020,2312.530612,...,1989.224490,1455.785714,0.501890,-0.139001,-0.480728,36.240387,2.031053,0.752727,-0.366197,24499.889319
1,-73.971928,40.788875,NaN,12,1020.693878,917.367347,1091.816327,1059.428571,1475.663265,2415.785714,...,2091.010204,1566.336735,0.438804,-0.130050,-0.426567,36.220483,1.888255,0.658119,-0.313933,10574.872312
2,-73.967080,40.789080,NaN,4,970.545455,1154.525253,1277.585859,1368.838384,1500.555556,1720.090909,...,1907.929293,1672.373737,0.134053,0.031153,-0.167759,35.535937,0.786147,0.201048,-0.197878,403763.833896
3,-73.972550,40.789082,NaN,12,1204.210000,1551.800000,1713.960000,1859.680000,2032.640000,2282.490000,...,2508.810000,2228.220000,0.116974,0.032179,-0.157001,36.614672,0.657649,0.175440,-0.188230,5144.297626
4,-73.969697,40.787953,NaN,9,1178.479592,1299.132653,1430.418367,1444.836735,1668.214286,2285.122449,...,2385.000000,1968.979592,0.270724,-0.027037,-0.275365,34.665793,1.857094,0.406035,-0.250191,222084.003584


In [17]:
validation_concat1.shape

(1040, 23)

## 5. Water feature 

In [18]:
train_water = pd.read_csv("/kaggle/input/new-water-750m/train_df_with_water (14).csv")
train_water.head()

,Longitude,Latitude,datetime,UHI Index,water
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,0.0
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,0.0
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,0.0
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,0.0
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,0.0


In [19]:
validation_water = pd.read_csv("/kaggle/input/new-water-750m/valid_df_with_water (14).csv")
validation_water.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,water
0,-73.971665,40.788763,NaN,32491.027812
1,-73.971928,40.788875,NaN,18151.225888
2,-73.967080,40.789080,NaN,313644.561414
3,-73.972550,40.789082,NaN,0.000000
4,-73.969697,40.787953,NaN,175156.542666


In [20]:
train_water = train_water.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat2 = pd.concat([train_concat1, train_water], axis = 1)
train_concat2.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,992.949495,969.676768,1130.010101,1223.454545,1451.595960,...,1630.909091,0.214126,-0.000035,-0.251691,38.009627,0.850690,0.321138,-0.251659,185571.898005,0.0
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,988.989796,949.540816,1101.928571,1191.387755,1428.622449,...,1588.693878,0.233759,-0.014216,-0.270303,38.009627,0.933826,0.350582,-0.257075,189946.352368,0.0
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,967.108911,913.376238,1068.574257,1140.584158,1387.514851,...,1514.772277,0.271812,-0.041667,-0.301735,37.939344,1.071730,0.407653,-0.263379,194989.481631,0.0
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,969.100000,920.130000,1076.030000,1139.090000,1374.240000,...,1477.340000,0.278584,-0.052145,-0.304636,37.692563,1.125919,0.417810,-0.256567,200157.261231,0.0
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,964.711340,919.000000,1076.845361,1127.010309,1361.938144,...,1451.185567,0.289269,-0.059123,-0.309990,37.712901,1.197662,0.433835,-0.255551,204423.999904,0.0


In [21]:
train_concat2.shape

(11229, 25)

In [22]:
validation_water = validation_water.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat2 = pd.concat([validation_concat1, validation_water], axis = 1)
validation_concat2.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,B12,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water
0,-73.971665,40.788763,NaN,12,927.510204,760.561224,922.836735,872.755102,1302.051020,2312.530612,...,1455.785714,0.501890,-0.139001,-0.480728,36.240387,2.031053,0.752727,-0.366197,24499.889319,32491.027812
1,-73.971928,40.788875,NaN,12,1020.693878,917.367347,1091.816327,1059.428571,1475.663265,2415.785714,...,1566.336735,0.438804,-0.130050,-0.426567,36.220483,1.888255,0.658119,-0.313933,10574.872312,18151.225888
2,-73.967080,40.789080,NaN,4,970.545455,1154.525253,1277.585859,1368.838384,1500.555556,1720.090909,...,1672.373737,0.134053,0.031153,-0.167759,35.535937,0.786147,0.201048,-0.197878,403763.833896,313644.561414
3,-73.972550,40.789082,NaN,12,1204.210000,1551.800000,1713.960000,1859.680000,2032.640000,2282.490000,...,2228.220000,0.116974,0.032179,-0.157001,36.614672,0.657649,0.175440,-0.188230,5144.297626,0.000000
4,-73.969697,40.787953,NaN,9,1178.479592,1299.132653,1430.418367,1444.836735,1668.214286,2285.122449,...,1968.979592,0.270724,-0.027037,-0.275365,34.665793,1.857094,0.406035,-0.250191,222084.003584,175156.542666


In [23]:
validation_concat2.shape

(1040, 24)

## 6. Road Density

In [24]:
train_road = pd.read_csv("/kaggle/input/new-road-density-450m/train_df_with_road_density.csv")
train_road.head()

,Longitude,Latitude,datetime,UHI Index,road_density
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,7205.206029
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,7077.541465
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,6963.614210
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,6844.866848
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,6739.519840


In [25]:
validation_road = pd.read_csv("/kaggle/input/new-road-density-450m/valid_df_with_road_density.csv")
validation_road.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,road_density
0,-73.971665,40.788763,NaN,8118.143055
1,-73.971928,40.788875,NaN,8499.583079
2,-73.967080,40.789080,NaN,4891.963542
3,-73.972550,40.789082,NaN,8907.302832
4,-73.969697,40.787953,NaN,6674.437137


In [26]:
train_road = train_road.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat3 = pd.concat([train_concat2, train_road], axis = 1)
train_concat3.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water,road_density
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,992.949495,969.676768,1130.010101,1223.454545,1451.595960,...,0.214126,-0.000035,-0.251691,38.009627,0.850690,0.321138,-0.251659,185571.898005,0.0,7205.206029
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,988.989796,949.540816,1101.928571,1191.387755,1428.622449,...,0.233759,-0.014216,-0.270303,38.009627,0.933826,0.350582,-0.257075,189946.352368,0.0,7077.541465
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,967.108911,913.376238,1068.574257,1140.584158,1387.514851,...,0.271812,-0.041667,-0.301735,37.939344,1.071730,0.407653,-0.263379,194989.481631,0.0,6963.614210
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,969.100000,920.130000,1076.030000,1139.090000,1374.240000,...,0.278584,-0.052145,-0.304636,37.692563,1.125919,0.417810,-0.256567,200157.261231,0.0,6844.866848
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,964.711340,919.000000,1076.845361,1127.010309,1361.938144,...,0.289269,-0.059123,-0.309990,37.712901,1.197662,0.433835,-0.255551,204423.999904,0.0,6739.519840


In [27]:
train_concat3.shape

(11229, 26)

In [28]:
validation_road = validation_road.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat3 = pd.concat([validation_concat2, validation_road], axis = 1)
validation_concat3.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,NDVI,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water,road_density
0,-73.971665,40.788763,NaN,12,927.510204,760.561224,922.836735,872.755102,1302.051020,2312.530612,...,0.501890,-0.139001,-0.480728,36.240387,2.031053,0.752727,-0.366197,24499.889319,32491.027812,8118.143055
1,-73.971928,40.788875,NaN,12,1020.693878,917.367347,1091.816327,1059.428571,1475.663265,2415.785714,...,0.438804,-0.130050,-0.426567,36.220483,1.888255,0.658119,-0.313933,10574.872312,18151.225888,8499.583079
2,-73.967080,40.789080,NaN,4,970.545455,1154.525253,1277.585859,1368.838384,1500.555556,1720.090909,...,0.134053,0.031153,-0.167759,35.535937,0.786147,0.201048,-0.197878,403763.833896,313644.561414,4891.963542
3,-73.972550,40.789082,NaN,12,1204.210000,1551.800000,1713.960000,1859.680000,2032.640000,2282.490000,...,0.116974,0.032179,-0.157001,36.614672,0.657649,0.175440,-0.188230,5144.297626,0.000000,8907.302832
4,-73.969697,40.787953,NaN,9,1178.479592,1299.132653,1430.418367,1444.836735,1668.214286,2285.122449,...,0.270724,-0.027037,-0.275365,34.665793,1.857094,0.406035,-0.250191,222084.003584,175156.542666,6674.437137


In [29]:
validation_concat3.shape

(1040, 25)

## 7. Building density 2

In [30]:
train_density2 = pd.read_csv("/kaggle/input/building-density2-taller-than-18m-500m/train_df_with_building_density2 (3).csv")
train_density2.head()

,Longitude,Latitude,datetime,UHI Index,building_density2
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,51
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,51
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,50
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,48
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,46


In [31]:
validation_density2 = pd.read_csv("/kaggle/input/building-density2-taller-than-18m-500m/valid_df_with_building_density2 (3).csv")
validation_density2.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,building_density2
0,-73.971665,40.788763,NaN,348
1,-73.971928,40.788875,NaN,351
2,-73.967080,40.789080,NaN,187
3,-73.972550,40.789082,NaN,333
4,-73.969697,40.787953,NaN,312


In [32]:
train_density2 = train_density2.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat4 = pd.concat([train_concat3, train_density2], axis = 1)
train_concat4.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water,road_density,building_density2
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,992.949495,969.676768,1130.010101,1223.454545,1451.595960,...,-0.000035,-0.251691,38.009627,0.850690,0.321138,-0.251659,185571.898005,0.0,7205.206029,51
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,988.989796,949.540816,1101.928571,1191.387755,1428.622449,...,-0.014216,-0.270303,38.009627,0.933826,0.350582,-0.257075,189946.352368,0.0,7077.541465,51
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,967.108911,913.376238,1068.574257,1140.584158,1387.514851,...,-0.041667,-0.301735,37.939344,1.071730,0.407653,-0.263379,194989.481631,0.0,6963.614210,50
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,969.100000,920.130000,1076.030000,1139.090000,1374.240000,...,-0.052145,-0.304636,37.692563,1.125919,0.417810,-0.256567,200157.261231,0.0,6844.866848,48
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,964.711340,919.000000,1076.845361,1127.010309,1361.938144,...,-0.059123,-0.309990,37.712901,1.197662,0.433835,-0.255551,204423.999904,0.0,6739.519840,46


In [33]:
train_concat4.shape

(11229, 27)

In [34]:
validation_density2 = validation_density2.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat4 = pd.concat([validation_concat3, validation_density2], axis = 1)
validation_concat4.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water,road_density,building_density2
0,-73.971665,40.788763,NaN,12,927.510204,760.561224,922.836735,872.755102,1302.051020,2312.530612,...,-0.139001,-0.480728,36.240387,2.031053,0.752727,-0.366197,24499.889319,32491.027812,8118.143055,348
1,-73.971928,40.788875,NaN,12,1020.693878,917.367347,1091.816327,1059.428571,1475.663265,2415.785714,...,-0.130050,-0.426567,36.220483,1.888255,0.658119,-0.313933,10574.872312,18151.225888,8499.583079,351
2,-73.967080,40.789080,NaN,4,970.545455,1154.525253,1277.585859,1368.838384,1500.555556,1720.090909,...,0.031153,-0.167759,35.535937,0.786147,0.201048,-0.197878,403763.833896,313644.561414,4891.963542,187
3,-73.972550,40.789082,NaN,12,1204.210000,1551.800000,1713.960000,1859.680000,2032.640000,2282.490000,...,0.032179,-0.157001,36.614672,0.657649,0.175440,-0.188230,5144.297626,0.000000,8907.302832,333
4,-73.969697,40.787953,NaN,9,1178.479592,1299.132653,1430.418367,1444.836735,1668.214286,2285.122449,...,-0.027037,-0.275365,34.665793,1.857094,0.406035,-0.250191,222084.003584,175156.542666,6674.437137,312


In [35]:
validation_concat4.shape

(1040, 26)

# Select features 

In [36]:
features = ['B01','B05','B07','B12','LST','density','water','green_space','road_density','building_density2']
train_df = train_concat4[features + ["UHI Index"]]
train_df

,B01,B05,B07,B12,LST,density,water,green_space,road_density,building_density2,UHI Index
0,992.949495,1451.595960,1941.626263,1630.909091,38.009627,8,0.000000,185571.898005,7205.206029,51,1.030289
1,988.989796,1428.622449,1950.183673,1588.693878,38.009627,8,0.000000,189946.352368,7077.541465,51,1.030289
2,967.108911,1387.514851,1994.138614,1514.772277,37.939344,8,0.000000,194989.481631,6963.614210,50,1.023798
3,969.100000,1374.240000,2000.800000,1477.340000,37.692563,8,0.000000,200157.261231,6844.866848,48,1.023798
4,964.711340,1361.938144,2015.896907,1451.185567,37.712901,8,0.000000,204423.999904,6739.519840,46,1.021634
...,...,...,...,...,...,...,...,...,...,...,...
11224,397.555556,940.747475,3274.040404,1050.040404,30.401756,0,245597.036636,766445.994788,2089.511617,45,0.972470
11225,404.397959,954.183673,3278.693878,1061.979592,30.401756,0,247432.375481,765978.971688,2093.197935,46,0.972470
11226,410.404040,963.333333,3282.767677,1069.020202,30.401756,0,250909.526972,766132.688085,2093.613387,46,0.981124
11227,411.920000,969.010000,3282.460000,1071.060000,30.407851,0,252775.520282,766681.994497,2091.379207,46,0.981245


# Remove duplicates from training data

In [37]:
# Remove duplicate rows from the DataFrame based on specified columns and keep the first occurrence
for col in features:
    # Check if the value is a numpy array and has more than one dimension
    train_df[col] = train_df[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)

# Now remove duplicates
uhi_data = train_df.drop_duplicates(subset=features, keep='first')
uhi_data.head()

<ipython-input-37-cc52a2d8b8e0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[col] = train_df[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)


,B01,B05,B07,B12,LST,density,water,green_space,road_density,building_density2,UHI Index
0,992.949495,1451.595960,1941.626263,1630.909091,38.009627,8,0.0,185571.898005,7205.206029,51,1.030289
1,988.989796,1428.622449,1950.183673,1588.693878,38.009627,8,0.0,189946.352368,7077.541465,51,1.030289
2,967.108911,1387.514851,1994.138614,1514.772277,37.939344,8,0.0,194989.481631,6963.614210,50,1.023798
3,969.100000,1374.240000,2000.800000,1477.340000,37.692563,8,0.0,200157.261231,6844.866848,48,1.023798
4,964.711340,1361.938144,2015.896907,1451.185567,37.712901,8,0.0,204423.999904,6739.519840,46,1.021634


In [38]:
uhi_data.shape

(11229, 11)

In [39]:
# Resetting the index of the dataset
uhi_data=uhi_data.reset_index(drop=True)

In [40]:
uhi_data.isna().sum()

B01                  0
B05                  0
B07                  0
B12                  0
LST                  0
density              0
water                0
green_space          0
road_density         0
building_density2    0
UHI Index            0
dtype: int64

# Model Building

In [41]:
# Split the data into features (X) and target (y), and then into training and testing sets
X_train = uhi_data.drop(columns=['UHI Index']).values
y_train = uhi_data ['UHI Index'].values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0,random_state=42)

**Feature Scaling**

In [42]:
# Scale the training and test data using standardscaler
sc = RobustScaler()
X_train = sc.fit_transform(X_train)
#X_test = sc.transform(X_test)

**Model Training**

TabPFN

In [43]:
!pip install tabpfn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.9/124.9 kB 4.2 MB/s eta 0:00:00


In [44]:
from tabpfn import TabPFNRegressor

# Initialize the TabPFN model and name it best_pfn
best_pfn = TabPFNRegressor(device="cuda", ignore_pretraining_limits=True)  # Use "cuda" for GPU or "cpu" for CPU

# Fit the model
best_pfn.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/tabpfn/base.py:100: UserWarning: Downloading model to /root/.cache/tabpfn/tabpfn-v2-regressor.ckpt.
  model, bardist, config_ = load_model_criterion_config(


tabpfn-v2-regressor.ckpt:   0%|          | 0.00/44.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/37.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/tabpfn/regressor.py:426: UserWarning: Number of samples 11229 is greater than the maximum Number of samples 10000 supported by the model. You may see degraded performance.
  X, y, feature_names_in, n_features_in = validate_Xy_fit(


TabPFNRegressor(device='cuda', ignore_pretraining_limits=True)

**Model Evaluation**

In [45]:
# Evaluate the ensemble on the test set
#final_r2 = r2_score(y_test, best_pfn.predict(X_test))
#print(f"Final R² score: {final_r2:.4f}")

# Submission

In [46]:
validation_concat4.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,NDBI,NDWI,LST,EVI,SAVI,MNDWI,green_space,water,road_density,building_density2
0,-73.971665,40.788763,NaN,12,927.510204,760.561224,922.836735,872.755102,1302.051020,2312.530612,...,-0.139001,-0.480728,36.240387,2.031053,0.752727,-0.366197,24499.889319,32491.027812,8118.143055,348
1,-73.971928,40.788875,NaN,12,1020.693878,917.367347,1091.816327,1059.428571,1475.663265,2415.785714,...,-0.130050,-0.426567,36.220483,1.888255,0.658119,-0.313933,10574.872312,18151.225888,8499.583079,351
2,-73.967080,40.789080,NaN,4,970.545455,1154.525253,1277.585859,1368.838384,1500.555556,1720.090909,...,0.031153,-0.167759,35.535937,0.786147,0.201048,-0.197878,403763.833896,313644.561414,4891.963542,187
3,-73.972550,40.789082,NaN,12,1204.210000,1551.800000,1713.960000,1859.680000,2032.640000,2282.490000,...,0.032179,-0.157001,36.614672,0.657649,0.175440,-0.188230,5144.297626,0.000000,8907.302832,333
4,-73.969697,40.787953,NaN,9,1178.479592,1299.132653,1430.418367,1444.836735,1668.214286,2285.122449,...,-0.027037,-0.275365,34.665793,1.857094,0.406035,-0.250191,222084.003584,175156.542666,6674.437137,312


In [47]:
submission_val_data = validation_concat4[features]
submission_val_data.head()

,B01,B05,B07,B12,LST,density,water,green_space,road_density,building_density2
0,927.510204,1302.051020,2593.826531,1455.785714,36.240387,12,32491.027812,24499.889319,8118.143055,348
1,1020.693878,1475.663265,2705.061224,1566.336735,36.220483,12,18151.225888,10574.872312,8499.583079,351
2,970.545455,1500.555556,1848.050505,1672.373737,35.535937,4,313644.561414,403763.833896,4891.963542,187
3,1204.210000,2032.640000,2406.940000,2228.220000,36.614672,12,0.000000,5144.297626,8907.302832,333
4,1178.479592,1668.214286,2559.214286,1968.979592,34.665793,9,175156.542666,222084.003584,6674.437137,312


In [48]:
# Feature Scaling 
submission_val_data = submission_val_data.values
transformed_submission_data = sc.transform(submission_val_data)

In [49]:
#Making predictions
final_predictions = best_pfn.predict(transformed_submission_data)
final_prediction_series = pd.Series(final_predictions)

In [50]:
#Combining the results into dataframe
submission_df = pd.DataFrame({'Longitude':validation_concat['Longitude'].values, 'Latitude':validation_concat['Latitude'].values, 'UHI Index':final_prediction_series.values})

In [51]:
#Displaying the sample submission dataframe
submission_df.head()

,Longitude,Latitude,UHI Index
0,-73.971665,40.788763,0.963600
1,-73.971928,40.788875,0.963996
2,-73.967080,40.789080,0.962031
3,-73.972550,40.789082,0.963156
4,-73.969697,40.787953,0.958869


In [52]:
#Dumping the predictions into a csv file.
submission_df.to_csv("submission.csv",index = False)